#### Portfolio Analytics
- Created: 2021.02.07

In [1]:
# import libraries
from library_functions import compute_expected_return
from library_functions import compute_var_return
from library_functions import compute_covariance
from library_functions import compute_correlation_coefficient

import pandas as pd
import numpy as np
from datetime import date, datetime

In [2]:
# Import Data
equity_data = pd.read_excel("Output/Equity_Data_2021-01-18.xlsx", sheet_name="Equity_Data", index_col="Date")
equity_data.head()

,SPY,AAPL,INTC
Date,,,
2007-03-01,140.509995,3.109286,19.590000
2007-03-02,138.669998,3.050357,19.219999
2007-03-05,137.350006,3.082857,19.110001
2007-03-06,139.699997,3.149643,19.400000
2007-03-07,139.559998,3.132857,19.120001


In [3]:
# Compute matrix of log returns
# pre-processing: fill any 'NA' values with the prior non-NA value
equity_data.fillna(method='ffill', inplace=True)

# Compute returns matrix
returns_matrix = pd.DataFrame(index=equity_data.index)
for ticker in equity_data.columns:
    returns_matrix[ticker] = np.log(equity_data[ticker].shift(1) / equity_data[ticker])

returns_matrix.dropna(axis=0, inplace=True) # drop na values
returns_matrix.index = [datetime.strptime(dt, '%Y-%M-%d').date() 
                            for dt in returns_matrix.index]
returns_matrix.head()

,SPY,AAPL,INTC
2007-01-02,0.013182,0.019134,0.019068
2007-01-05,0.009565,-0.010598,0.005740
2007-01-06,-0.016965,-0.021432,-0.015061
2007-01-07,0.001003,0.005344,0.014538
2007-01-08,-0.008420,-0.003187,-0.005737


In [4]:
# compute expected returns for each asset
expected_returns = np.array([compute_expected_return(returns_matrix[ticker]) for ticker in returns_matrix.columns])
expected_returns

array([-0.02519678, -0.09562935, -0.03408655])

In [5]:
# compute expected returns for each asset
variance_returns = np.array([compute_var_return(returns_matrix[ticker], expected_returns[i]) for (i,ticker) in enumerate(returns_matrix.columns)])
variance_returns

array([0.07724643, 0.93536465, 0.14748382])

In [6]:
# Average portfolio return
average_portfolio_return = np.average(expected_returns)
print("Average portfolio return: {}".format(average_portfolio_return))

Average portfolio return: -0.05163756391934097


In [7]:
# Average return of S&P (using SPY ETF as a proxy)
average_market_return = expected_returns[0]
print("Average market return: {}".format(average_market_return))

Average market return: -0.025196781880489166


In [8]:
# Average portfolio volatility (standard deviation)
average_portfolio_stdev = np.sqrt(np.average(variance_returns))
print("Average portfolio volatility: {}".format(average_portfolio_stdev))

Average portfolio volatility: 0.6218507045991832


In [9]:
# Portfolio Sharpe Ratio (using an arbitrary risk-free rate)
average_risk_free_rate = 0.05 # arbitrary set
portfolio_sharpe_ratio = (average_portfolio_return - average_risk_free_rate) / average_portfolio_stdev
print("Portfolio Sharpe Ratio: {}".format(portfolio_sharpe_ratio))

Portfolio Sharpe Ratio: -0.1634436741289084


In [10]:
# Correlation matrix
num_assets = len(returns_matrix.columns)
cov_matrix = np.zeros(num_assets**2).reshape(num_assets,num_assets)

for i in range(1, num_assets):
    for j in range(1, num_assets):
        if i == j:
            cov_matrix[i,j] = compute_var_return(returns_matrix.iloc[:,i], expected_returns[i-1])
        else:
            cov_matrix[i,j] = compute_covariance(returns_matrix.iloc[:,i], expected_returns[i-1],
                                                 returns_matrix.iloc[:,j], expected_returns[j-1])

# covariance matrix
cov_matrix = pd.DataFrame(cov_matrix, index=returns_matrix.columns, columns=returns_matrix.columns)
cov_matrix.style.background_gradient(cmap='coolwarm')

,SPY,AAPL,INTC
SPY,0.000000,0.000000,0.000000
AAPL,0.000000,0.097565,0.002490
INTC,0.000000,0.002490,0.941844


In [11]:
# Correlation coefficient matrix

# Alternative method: using corr() method
#c = returns_matrix.corr(method='pearson')
#c.style.background_gradient(cmap='coolwarm')

# compute correlation matrix
cor_matrix = np.zeros(num_assets**2).reshape(num_assets,num_assets)

for i in range(0, num_assets):
    for j in range(0, num_assets):
        cov_returns = cov_matrix.iloc[i,j]
        stdev_i = np.sqrt(variance_returns[i-1])
        stdev_j = np.sqrt(variance_returns[j-1])
        cor_matrix[i,j] = compute_correlation_coefficient(cov_returns, stdev_i, stdev_j, i, j)

cor_matrix = pd.DataFrame(cor_matrix, index=returns_matrix.columns, columns=returns_matrix.columns)
cor_matrix.style.background_gradient(cmap='coolwarm')

,SPY,AAPL,INTC
SPY,1.000000,0.000000,0.000000
AAPL,0.000000,1.000000,0.009265
INTC,0.000000,0.009265,1.000000


In [12]:
performance_results = pd.DataFrame({'Expected Return' : expected_returns,
                                    'Variance' : variance_returns}, index=returns_matrix.columns)
performance_results.index.name = 'Asset'
performance_results

,Expected Return,Variance
Asset,,
SPY,-0.025197,0.077246
AAPL,-0.095629,0.935365
INTC,-0.034087,0.147484


##### Export Output

In [13]:
# export to excel
with pd.ExcelWriter("Output/Portfolio_Analysis_{}.xlsx".format(date.today())) as output:
    returns_matrix.to_excel(output, sheet_name="Log_Returns")
    performance_results.to_excel(output, sheet_name="Performance")
    cov_matrix.style.background_gradient(cmap='coolwarm').to_excel(output, sheet_name="Covariance Matrix")
    cor_matrix.style.background_gradient(cmap='coolwarm').to_excel(output, sheet_name="Correlation Matrix")